<a href="https://colab.research.google.com/github/y-ack/mamba-comparison/blob/main/conda-mamba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## install miniconda
!wget -nv https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh

# install micromamba from conda
!conda install -y micromamba -c conda-forge
!export MAMBA_ROOT_PREFIX=/usr/local

# get the environment spec
!wget -nv https://raw.githubusercontent.com/czbiohub-sf/CRISPRflow/main/environment.yml -O CRISPRflow_env.yml

exec_times = [None, None, None, None]

In [2]:
!perl -p0e 's/  - default\n  - defaults/  - nodefaults/' CRISPRflow_env.yml > CRISPRflow_env_new.yml
!diff CRISPRflow_env.yml CRISPRflow_env_new.yml

5,6c5
<   - default
<   - defaults
---
>   - nodefaults


In [3]:
!micromamba remove -r /usr/local -q -y --name CRISPRflow --all > /dev/null
!conda clean -q -y --all > /dev/null

In [ ]:
exec_times[0] = %timeit -o -n 1 -r 1 !conda env create -y -f CRISPRflow_env.yml -n CRISPRflow

In [5]:
!micromamba remove -r /usr/local -q -y --name CRISPRflow --all > /dev/null
!conda clean -q -y --all > /dev/null

In [ ]:
exec_times[1] = %timeit -o -n 1 -r 1 !conda env create -y -f CRISPRflow_env_new.yml -n CRISPRflow

In [7]:
!micromamba remove -r /usr/local -q -y --name CRISPRflow --all > /dev/null
!micromamba clean -q -y --all > /dev/null

In [ ]:
exec_times[2] = %timeit -o -n 1 -r 1 !micromamba env create -y -f CRISPRflow_env.yml -n CRISPRflow

In [9]:
!micromamba remove -r /usr/local -q -y --name CRISPRflow --all > /dev/null
!micromamba clean -q -y --all > /dev/null

In [ ]:
exec_times[3] = %timeit -o -n 1 -r 1 !micromamba env create -y -f CRISPRflow_env_new.yml -n CRISPRflow

In [18]:
import pandas as pd
import numpy as np

exec_times = np.vectorize(lambda t: np.mean(t.all_runs))(exec_times)
exec_times = exec_times.reshape((2, 2))

df = pd.DataFrame(data=exec_times, index=pd.Index(['Miniconda', 'micromamba']),
                  columns=pd.Index(['default/defaults', 'nodefaults'], name='Default Channels:'))
df.style

Default Channels:,default/defaults,nodefaults
Miniconda,293.579338,259.245121
micromamba,47.856322,36.200433
